# Proyecto de Marketing Analytics – Pipeline completo

In [ ]:
import pandas as pd 
import numpy as np
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

# === Ruta al archivo (ajusta si hace falta) ===
INPUT_PATH = "/datasets/customer_data.csv"  # cambia por tu ruta

# Opcionales útiles
READ_KW = dict(low_memory=False)  
pd.set_option("display.max_columns", 120)
print(f"Archivo de entrada: {INPUT_PATH}")

## Paso 2 — Cargar y Revisar
**Checklist:**
- `pd.read_csv()`
- `df.info()`, `df.head()`
- Revisar columnas clave:
  - ID cliente: `CustomerID`
  - Fecha compra: `OrderDate` / `InvoiceDate`
  - Monto: `Total` / `Price` / `Sales` / `Amount`
  - Orden: `OrderID` / `InvoiceNo`

In [ ]:
# Carga
df_raw = pd.read_csv(INPUT_PATH, **READ_KW)

# Revisión
print("=== INFO INICIAL DEL CSV ===")
print(df_raw.info())
display(df_raw.head(10))

# Copia de trabajo
df = df_raw.copy()

## Paso 3 — Limpieza
**Acciones:**
- Normalizar nombres de columnas
- Resolver columnas clave (detección flexible)
- Convertir fechas a `datetime`
- Eliminar duplicados
- Manejar nulos en claves
- Eliminar montos negativos / cancelaciones si aplica
- Derivar `order_month` (si hay fecha)

In [ ]:
# 3.1 Normalizar columnas
def normalize(col):
    return (col.strip().lower()
            .replace(" ", "_")
            .replace("-", "_")
            .replace(".", "_")
            .replace("/", "_"))

df.columns = [normalize(c) for c in df.columns]

# 3.2 Resolver columnas clave (flexible)
candidates = {
    "customer_id": ["customerid", "customer_id", "id_cliente", "cliente_id", "idcustomer", "id_client", "id"],
    "order_date": ["orderdate", "order_date", "invoice_date", "invoicedate", "fecha_compra", "date", "fecha", "order_datetime"],
    "amount": ["total", "price", "sales", "monto", "amount", "importe", "revenue", "line_total", "subtotal", "sales_amount", "unit_price"],
    "order_id": ["orderid", "order_id", "invoice_no", "invoiceno", "invoice", "order_no", "orden", "order_number", "invoice_number"],
    "product": ["product", "product_name", "item", "sku", "descripcion", "description", "product_title"],
    "category": ["category", "categoria", "product_category", "segment", "dept", "department"]
}
def pick_column(possible, available):
    for name in possible:
        if name in available:
            return name
    return None

resolved = {k: pick_column(v, set(df.columns)) for k, v in candidates.items()}
print("Columnas detectadas:", resolved)

# 3.3 Fechas a datetime y montos a numérico
if resolved.get("order_date"):
    df[resolved["order_date"]] = pd.to_datetime(df[resolved["order_date"]], errors="coerce")
if resolved.get("amount"):
    df[resolved["amount"]] = pd.to_numeric(df[resolved["amount"]], errors="coerce")

# 3.4 Eliminar duplicados
before = len(df)
df = df.drop_duplicates()
print("Duplicados eliminados:", before - len(df))

# 3.5 Manejo de nulos en claves
mask = pd.Series(True, index=df.index)
for key in ["customer_id", "order_date", "amount"]:
    col = resolved.get(key)
    if col:
        mask &= df[col].notna()
print("Filas eliminadas por nulos en claves:", int((~mask).sum()))
df = df[mask].copy()

# 3.6 Eliminar montos <= 0 (si aplica)
if resolved.get("amount"):
    nonpos = int((df[resolved["amount"]] <= 0).sum())
    df = df[df[resolved["amount"]] > 0].copy()
    print("Filas eliminadas por montos <= 0:", nonpos)

# 3.7 Derivar order_month
if resolved.get("order_date"):
    df["order_month"] = df[resolved["order_date"]].values.astype("datetime64[M]")

display(df.head(10))

## Paso 4 — Detección del tipo de dataset
- **Transaccional**: tiene `order_date` y `amount`
- **Nivel cliente (tipo UCI)**: tiene `recency` y/o columnas `Mnt*` / `Num*Purchases`

En nivel cliente, se construyen proxies:
- `monetary_proxy` (suma de `Mnt*`)
- `frequency_proxy` (suma de `NumWebPurchases + NumStorePurchases + NumCatalogPurchases`)

In [ ]:
has_order_date = resolved.get("order_date") is not None
has_amount = resolved.get("amount") is not None
is_transactional = has_order_date and has_amount

has_recency = "recency" in df.columns
mnt_cols = [c for c in df.columns if c.startswith("mnt")]
freq_cols = [c for c in df.columns if c in ["numwebpurchases","numcatalogpurchases","numstorepurchases"]]

dataset_type = "transactional" if is_transactional else ("customer_level" if has_recency or mnt_cols or freq_cols else "unknown")
print("Tipo de dataset:", dataset_type)

# En caso 'customer_level' (tipo UCI): construir proxies
if dataset_type == "customer_level":
    if mnt_cols:
        for c in mnt_cols:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df["monetary_proxy"] = df[mnt_cols].sum(axis=1, skipna=True)
    if freq_cols:
        for c in freq_cols:
            df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)
        df["frequency_proxy"] = df[freq_cols].sum(axis=1)
    if has_recency:
        df["recency"] = pd.to_numeric(df["recency"], errors="coerce")

import os

CLEAN_PATH = "/mnt/data/customer_data_clean.csv"
os.makedirs(os.path.dirname(CLEAN_PATH), exist_ok=True)
df.to_csv(CLEAN_PATH, index=False)
print(f"Guardado: {CLEAN_PATH}  | Filas: {len(df)}  | Columnas: {len(df.columns)}")

## Paso 6 — KPIs mínimos (Marketing/BI)
Calcular y guardar:
- Revenue total y mensual (si hay fechas)
- Orders total y mensual (si hay `order_id`; si no, filas)
- Customers únicos (mensual)
- **AOV** = revenue / orders
- **Repeat rate** = % clientes con 2+ compras
- Top categorías/productos (si existen)

In [ ]:
# KPIs
if dataset_type == "transactional":
    revenue_total = df[resolved["amount"]].sum()
    orders_total = df[resolved["order_id"]].nunique() if resolved.get("order_id") else len(df)
    total_customers = df[resolved["customer_id"]].nunique() if resolved.get("customer_id") else np.nan
    AOV_total = (revenue_total / orders_total) if orders_total else np.nan

    revenue_by_month = df.groupby("order_month")[resolved["amount"]].sum().rename("revenue")
    orders_by_month = (df.groupby("order_month")[resolved["order_id"]].nunique().rename("orders")
                       if resolved.get("order_id")
                       else df.groupby("order_month").size().rename("orders"))
    customers_by_month = (df.groupby("order_month")[resolved["customer_id"]].nunique().rename("unique_customers")
                          if resolved.get("customer_id") else None)

    monthly_kpis = pd.concat([revenue_by_month, orders_by_month, customers_by_month], axis=1)
    monthly_kpis["AOV"] = monthly_kpis["revenue"] / monthly_kpis["orders"]

    # Repeat rate
    if resolved.get("customer_id"):
        if resolved.get("order_id"):
            freq_by_client = df.groupby(resolved["customer_id"])[resolved["order_id"]].nunique()
        else:
            freq_by_client = df.groupby(resolved["customer_id"]).size()
        repeat_rate = (freq_by_client.ge(2).sum() / freq_by_client.index.nunique()) if freq_by_client.size else np.nan
    else:
        repeat_rate = np.nan

    kpi_totals = pd.DataFrame({
        "revenue_total": [revenue_total],
        "orders_total": [orders_total],
        "AOV_total": [AOV_total],
        "unique_customers_total": [total_customers],
        "repeat_rate": [repeat_rate]
    })

    MONTHLY_KPIS_PATH = "kpis_monthly.csv"
    TOTALS_KPIS_PATH = "kpis_totals.csv"
    monthly_kpis.to_csv(MONTHLY_KPIS_PATH)
    kpi_totals.to_csv(TOTALS_KPIS_PATH, index=False)

    display(monthly_kpis.head(12))
    display(kpi_totals)

else:
    # Nivel cliente (proxies)
    unique_customers_total = len(df)
    revenue_total = df["monetary_proxy"].sum(skipna=True) if "monetary_proxy" in df.columns else np.nan
    orders_total = df["frequency_proxy"].sum(skipna=True) if "frequency_proxy" in df.columns else np.nan
    AOV_total = (revenue_total / orders_total) if (pd.notna(revenue_total) and pd.notna(orders_total) and orders_total>0) else np.nan
    repeat_rate = (df["frequency_proxy"] >= 2).mean() if "frequency_proxy" in df.columns else np.nan

    kpi_totals = pd.DataFrame({
        "unique_customers_total": [unique_customers_total],
        "revenue_total_proxy": [revenue_total],
        "orders_total_proxy": [orders_total],
        "AOV_total_proxy": [AOV_total],
        "repeat_rate_proxy": [repeat_rate]
    })

    TOTALS_KPIS_PATH = "kpis_totals_proxy.csv"
    kpi_totals.to_csv(TOTALS_KPIS_PATH, index=False)
    display(kpi_totals)

# Top categorías/productos
top_products = pd.DataFrame()
top_categories = pd.DataFrame()
if dataset_type == "transactional":
    if resolved.get("product") and resolved.get("amount"):
        top_products = (df.groupby(resolved["product"])[resolved["amount"]].sum()
                          .sort_values(ascending=False).head(20).reset_index()
                          .rename(columns={resolved["product"]:"product", resolved["amount"]:"revenue"}))
        TOP_PROD_PATH = "top_products.csv"
        top_products.to_csv(TOP_PROD_PATH, index=False)
        display(top_products)

    if resolved.get("category") and resolved.get("amount"):
        top_categories = (df.groupby(resolved["category"])[resolved["amount"]].sum()
                            .sort_values(ascending=False).head(20).reset_index()
                            .rename(columns={resolved["category"]:"category", resolved["amount"]:"revenue"}))
        TOP_CAT_PATH = "top_categories.csv"
        top_categories.to_csv(TOP_CAT_PATH, index=False)
        display(top_categories)
else:
    mnt_list = [c for c in df.columns if c.startswith("mnt")]
    if mnt_list:
        sums = df[mnt_list].sum().sort_values(ascending=False).reset_index()
        top_categories = sums.rename(columns={"index":"category", 0:"revenue"})
        top_categories["category"] = top_categories["category"].str.replace("^mnt","",regex=True)
        TOP_CAT_PATH = "top_categories_proxy.csv"
        top_categories.to_csv(TOP_CAT_PATH, index=False)
        display(top_categories)

## Paso 7 — Segmentación (RFM)
- Recency: días desde última compra (o `recency` ya incluida)
- Frequency: número de compras (por `order_id` o `frequency_proxy`)
- Monetary: gasto total (`amount` o `monetary_proxy`)
- Scores 1–5 por cuantiles (Recency invertida)
- Segmentos: Champions, Loyal, Potential Loyalist, At Risk, Hibernating

In [ ]:
rfm = pd.DataFrame()

if dataset_type == "transactional" and resolved.get("customer_id") and resolved.get("order_date"):
    max_date = df[resolved["order_date"]].max()
    if resolved.get("order_id"):
        freq_per_customer = df.groupby(resolved["customer_id"])[resolved["order_id"]].nunique().rename("frequency")
    else:
        freq_per_customer = df.groupby(resolved["customer_id"]).size().rename("frequency")
    monetary_per_customer = (df.groupby(resolved["customer_id"])[resolved["amount"]].sum().rename("monetary")
                             if resolved.get("amount") else df.groupby(resolved["customer_id"]).size().rename("monetary"))
    recency_per_customer = (max_date - df.groupby(resolved["customer_id"])[resolved["order_date"]].max()).dt.days.rename("recency")
    rfm = pd.concat([recency_per_customer, freq_per_customer, monetary_per_customer], axis=1).reset_index()
    rfm = rfm.rename(columns={resolved["customer_id"]:"customer_id"})

elif dataset_type == "customer_level":
    id_col = resolved.get("customer_id") if resolved.get("customer_id") else ("id" if "id" in df.columns else None)
    base_cols = ["recency","frequency_proxy","monetary_proxy"]
    if all([c in df.columns for c in base_cols]):
        rfm = df[base_cols].copy()
        rfm.insert(0, "customer_id", df[id_col].values if id_col else np.arange(len(rfm)))
        rfm = rfm.rename(columns={"frequency_proxy":"frequency","monetary_proxy":"monetary"})

if not rfm.empty:
    def quantile_scores(series, q=5, reverse=False):
        if series.nunique() <= 1:
            return pd.Series(np.full(len(series), 3), index=series.index)
        try:
            cats = pd.qcut(series.rank(method="first"), q, labels=False, duplicates="drop") + 1
        except ValueError:
            cats = pd.cut(series.rank(method="first"), q, labels=False) + 1
        scores = cats.astype(int)
        if reverse:
            scores = (q + 1) - scores
        return scores

    rfm["R_score"] = quantile_scores(rfm["recency"], q=5, reverse=True)
    rfm["F_score"] = quantile_scores(rfm["frequency"], q=5, reverse=False)
    rfm["M_score"] = quantile_scores(rfm["monetary"], q=5, reverse=False)
    rfm["RFM_sum"] = rfm[["R_score","F_score","M_score"]].sum(axis=1)

    def segment_row(r, r_thr=4, f_thr=4, m_thr=4):
        if (r["R_score"] >= r_thr) and (r["F_score"] >= f_thr) and (r["M_score"] >= m_thr):
            return "Champions"
        if (r["F_score"] >= f_thr):
            return "Loyal"
        if (r["R_score"] >= r_thr) and (r["F_score"] >= 3):
            return "Potential Loyalist"
        if (r["R_score"] <= 2) and (r["F_score"] >= 2):
            return "At Risk"
        return "Hibernating"

    rfm["Segment"] = rfm.apply(segment_row, axis=1)

    RFM_PATH = "rfm_table.csv"
    rfm.to_csv(RFM_PATH, index=False)
    display(rfm.head(20))
else:
    print("No fue posible construir RFM con las columnas disponibles.")

## Paso 8 — Cohortes de retención (si hay fechas)
Variables:
- `cohort_month` = mes de primera compra del cliente
- `order_month` = mes de la compra
- `cohort_index` = meses desde la primera compra
Salida: **Matriz de retención (%)** por cohorte.

In [ ]:
if dataset_type == "transactional" and resolved.get("customer_id") and resolved.get("order_date"):
    temp = df[[resolved["customer_id"], resolved["order_date"]]].copy()
    temp["order_month"] = temp[resolved["order_date"]].values.astype("datetime64[M]")
    first_purchase = temp.groupby(resolved["customer_id"])["order_month"].min().rename("cohort_month")
    temp = temp.merge(first_purchase, left_on=resolved["customer_id"], right_index=True, how="left")

    def month_diff(end, start):
        return (end.year - start.year) * 12 + (end.month - start.month)

    temp["cohort_index"] = temp.apply(lambda r: month_diff(r["order_month"], r["cohort_month"]), axis=1)

    cohort_sizes = temp.groupby("cohort_month")[resolved["customer_id"]].nunique().rename("cohort_size")
    active = temp.groupby(["cohort_month","cohort_index"])[resolved["customer_id"]].nunique().rename("active_users").reset_index()
    cohort_pivot = active.pivot(index="cohort_month", columns="cohort_index", values="active_users").fillna(0)
    cohort_matrix = (cohort_pivot.divide(cohort_sizes, axis=0).round(4)) * 100

    COHORT_PATH = "cohort_retention_matrix.csv"
    cohort_matrix.to_csv(COHORT_PATH)
    display(cohort_matrix.head())
else:
    print("Cohortes no construidas: faltan fechas de compra transaccionales.")

## Paso 9 — Exportaciones y Resumen
Rutas de archivos generados para el dashboard (según tipo de dataset):
- `customer_data_clean.csv`
- `kpis_totals.csv` / `kpis_totals_proxy.csv`
- `kpis_monthly.csv`
- `top_products.csv` / `top_categories.csv` / `top_categories_proxy.csv`
- `rfm_table.csv`
- `cohort_retention_matrix.csv`

In [ ]:
outputs = {
    "clean_data": "/mnt/data/customer_data_clean.csv",
    "kpis_totals": "/mnt/data/kpis_totals.csv",
    "kpis_monthly": "/mnt/data/kpis_monthly.csv",
    "kpis_totals_proxy": "/mnt/data/kpis_totals_proxy.csv",
    "top_products": "/mnt/data/top_products.csv",
    "top_categories": "/mnt/data/top_categories.csv",
    "top_categories_proxy": "/mnt/data/top_categories_proxy.csv",
    "rfm_table": "/mnt/data/rfm_table.csv",
    "cohort_matrix": "/mnt/data/cohort_retention_matrix.csv"
}
for k,v in outputs.items():
    print(f"{k}: {v}  {'(existe)' if Path(v).exists() else ''}")

## Paso 10 — Gráfico: Revenue mensual
Si el dataset es transaccional, se grafica la serie mensual de revenue.
Si el dataset es nivel cliente, no hay fechas de compra; se omite y se informa.
Se guarda imagen en: `/mnt/data/revenue_mensual.png`

In [ ]:
import matplotlib.pyplot as plt

if dataset_type == "transactional" and "order_month" in df.columns and resolved.get("amount"):
    rev_month = df.groupby("order_month")[resolved["amount"]].sum()
    plt.figure()
    rev_month.plot(kind="line", title="Revenue mensual")
    plt.xlabel("Mes")
    plt.ylabel("Revenue")
    plt.tight_layout()
    plt.savefig("/mnt/data/revenue_mensual.png", dpi=150)
    plt.show()
    print("Imagen guardada en /mnt/data/revenue_mensual.png")
else:
    print("No se grafica revenue mensual: el dataset no es transaccional o no hay columnas de fecha/monto.")

## Paso 11 — Gráfico: Orders mensual / Distribución de frecuencia
Transaccional: gráfica de órdenes por mes.
Nivel cliente: no hay serie mensual, se muestra **histograma** de `frequency_proxy` para entender la distribución de compras.
Se guarda imagen en:
- `/mnt/data/orders_mensual.png` (transaccional)
- `/mnt/data/frequency_distribution.png` (nivel cliente)

In [ ]:
import matplotlib.pyplot as plt

if dataset_type == "transactional" and "order_month" in df.columns:
    if resolved.get("order_id"):
        orders_by_month = df.groupby("order_month")[resolved["order_id"]].nunique()
    else:
        orders_by_month = df.groupby("order_month").size()
    plt.figure()
    orders_by_month.plot(kind="bar", title="Órdenes mensuales")
    plt.xlabel("Mes")
    plt.ylabel("Órdenes")
    plt.tight_layout()
    plt.savefig("/mnt/data/orders_mensual.png", dpi=150)
    plt.show()
    print("Imagen guardada en /mnt/data/orders_mensual.png")
else:
    # Nivel cliente: histograma de frecuencia_proxy si existe
    if "frequency_proxy" in df.columns:
        plt.figure()
        df["frequency_proxy"].plot(kind="hist", bins=20, title="Distribución de frecuencia de compras (frequency_proxy)")
        plt.xlabel("Compras por cliente (proxy)")
        plt.ylabel("Frecuencia")
        plt.tight_layout()
        plt.savefig("/mnt/data/frequency_distribution.png", dpi=150)
        plt.show()
        print("Imagen guardada en /mnt/data/frequency_distribution.png")
    else:
        print("No hay 'frequency_proxy' disponible para graficar distribución en dataset de nivel cliente.")

## Paso 12 — Gráfico: Top categorías (Top 10)
Transaccional: suma de `amount` por `category` (si existe).
Nivel cliente: suma de columnas `Mnt*` y renombrado de categoría (ej. `MntWines` → `wines`).
Se guarda imagen en: `/mnt/data/top_categorias.png`

In [ ]:
import matplotlib.pyplot as plt

saved = False

if dataset_type == "transactional" and resolved.get("category") and resolved.get("amount"):
    top_cat = (df.groupby(resolved["category"])[resolved["amount"]]
                 .sum()
                 .sort_values(ascending=False)
                 .head(10)
                 .reset_index()
                 .rename(columns={resolved["category"]: "category", resolved["amount"]: "revenue"}))
    plt.figure()
    top_cat.set_index("category")["revenue"].plot(kind="bar", title="Top categorías (Top 10)")
    plt.xlabel("Categoría")
    plt.ylabel("Revenue")
    plt.tight_layout()
    plt.savefig("/mnt/data/top_categorias.png", dpi=150)
    plt.show()
    saved = True

elif dataset_type == "customer_level":
    mnt_cols_plot = [c for c in df.columns if c.startswith("mnt")]
    if mnt_cols_plot:
        sums = df[mnt_cols_plot].sum().sort_values(ascending=False).head(10).reset_index()
        sums.columns = ["category", "revenue"]
        sums["category"] = sums["category"].str.replace("^mnt", "", regex=True)
        plt.figure()
        sums.set_index("category")["revenue"].plot(kind="bar", title="Top categorías (Top 10, proxy)")
        plt.xlabel("Categoría")
        plt.ylabel("Revenue (proxy)")
        plt.tight_layout()
        plt.savefig("/mnt/data/top_categorias.png", dpi=150)
        plt.show()
        saved = True

if saved:
    print("Imagen guardada en /mnt/data/top_categorias.png")
else:
    print("No se generó gráfico de categorías: faltan columnas adecuadas (category/amount o Mnt*).")